In [1]:
import pandas as pd
import numpy as np
import glob
import os
import datetime

date_list = ['20201014', '20201015', '20201016', '20201019', '20201020']  
result = []
for d in date_list:
    date = d
    
    readPath = 'A:\\rawData'
    mdOrderLogPath = glob.glob(os.path.join(readPath, 'logs_%s_zs_92_01***'%date, 'mdOrderLog***.csv'))[-1]
    mdTradeLogPath = glob.glob(os.path.join(readPath, 'logs_%s_zs_92_01***'%date, 'mdTradeLog***.csv'))[-1]
    
    mdOrderLog = pd.read_csv(mdOrderLogPath)
    mdOrderLog['OrderType'] = mdOrderLog['OrderType'].astype(str)
    mdOrderLog = mdOrderLog[['clockAtArrival', 'sequenceNo', 'TransactTime', 'SecurityID', 'ApplSeqNum', 'Side',
                             'OrderType', 'Price', 'OrderQty']]
    
    mdTradeLog = pd.read_csv(mdTradeLogPath, encoding='utf-8')
    mdTradeLog['ExecType'] = mdTradeLog['ExecType'].astype(str)
    mdTradeLog['volumeThisUpdate'] = np.where(mdTradeLog['ExecType'] == 'F', mdTradeLog['TradeQty'], 0)
    mdTradeLog['cum_volume'] = mdTradeLog.groupby(['SecurityID'])['volumeThisUpdate'].cumsum()
    mdTradeLog = mdTradeLog[['clockAtArrival', 'sequenceNo', 'TransactTime', 'SecurityID', 'ApplSeqNum', 'cum_volume',
                             'ExecType', 'TradePrice', 'TradeQty', 'TradeMoney', 'BidApplSeqNum', 'OfferApplSeqNum']]
    mdTradeLog = mdTradeLog[mdTradeLog['ExecType'] == 'F']
    mdTradeLog = mdTradeLog[mdTradeLog['SecurityID'] < 600000]
    mdTradeLog = mdTradeLog[(mdTradeLog['TransactTime'] >= 92900000) & (mdTradeLog['TransactTime'] <= 145655000)]
    mdTradeLog['Aggressive_side'] = np.where(mdTradeLog['BidApplSeqNum'] > mdTradeLog['OfferApplSeqNum'], 1, 2)
    mdTradeLog['count1'] = np.nan
    mdTradeLog.loc[mdTradeLog['Aggressive_side'] == 1, 'count1'] = mdTradeLog[mdTradeLog['Aggressive_side'] == 1].groupby(['SecurityID', 'BidApplSeqNum']).cumcount()
    mdTradeLog.loc[mdTradeLog['Aggressive_side'] == 2, 'count1'] = mdTradeLog[mdTradeLog['Aggressive_side'] == 2].groupby(['SecurityID', 'OfferApplSeqNum']).cumcount()
    
#     mdTradeLog['count1'] = np.where(mdTradeLog['Aggressive_side'] == 1, mdTradeLog.groupby(['SecurityID', 'BidApplSeqNum']).cumcount(), 
#                                 mdTradeLog.groupby(['SecurityID', 'OfferApplSeqNum']).cumcount())

    mdTradeLog = mdTradeLog.sort_values(by='sequenceNo').reset_index(drop=True)
    mdTradeLog['index'] = mdTradeLog.groupby('SecurityID').cumcount()
    mdTradeLog['first'] = np.nan
    mdTradeLog.loc[mdTradeLog['Aggressive_side'] == 1, 'first'] = mdTradeLog[mdTradeLog['Aggressive_side'] == 1].groupby(['SecurityID', 'BidApplSeqNum'])['index'].transform('first')
    mdTradeLog.loc[mdTradeLog['Aggressive_side'] == 2, 'first'] = mdTradeLog[mdTradeLog['Aggressive_side'] == 2].groupby(['SecurityID', 'OfferApplSeqNum'])['index'].transform('first')
    mdTradeLog['sum'] = mdTradeLog['first'] + mdTradeLog['count1']
    test1 = mdTradeLog[(mdTradeLog['sum'] == mdTradeLog['index'])][mdTradeLog[(mdTradeLog['sum'] == mdTradeLog['index'])]['Aggressive_side'] == 1]
    test2 = mdTradeLog[(mdTradeLog['sum'] == mdTradeLog['index'])][mdTradeLog[(mdTradeLog['sum'] == mdTradeLog['index'])]['Aggressive_side'] == 2]
    del mdTradeLog
    test1 = pd.concat([test1.groupby(['SecurityID', 'BidApplSeqNum']).first().reset_index(),
                      test1.groupby(['SecurityID', 'BidApplSeqNum']).last().reset_index()])
    test2 = pd.concat([test2.groupby(['SecurityID', 'OfferApplSeqNum']).first().reset_index(),
                      test2.groupby(['SecurityID', 'OfferApplSeqNum']).last().reset_index()])    
    test1 = pd.merge(test1, mdOrderLog[['clockAtArrival', 'SecurityID', 'ApplSeqNum', 'sequenceNo']], left_on=['SecurityID', 'BidApplSeqNum'],
                    right_on=['SecurityID', 'ApplSeqNum'])
    test2 = pd.merge(test2, mdOrderLog[['clockAtArrival', 'SecurityID', 'ApplSeqNum', 'sequenceNo']], left_on=['SecurityID', 'OfferApplSeqNum'],
                    right_on=['SecurityID', 'ApplSeqNum'])
    del mdOrderLog
    test1['diff'] = test1['ApplSeqNum_x'] - test1['ApplSeqNum_y']
    test1['diff'] = test1.groupby(['SecurityID', 'BidApplSeqNum'])['diff'].transform('min')
    test2['diff'] = test2['ApplSeqNum_x'] - test2['ApplSeqNum_y']
    test2['diff'] = test2.groupby(['SecurityID', 'OfferApplSeqNum'])['diff'].transform('min')
    test1 = test1[test1['diff'] == 1].sort_values(by=['SecurityID', 'BidApplSeqNum', 'sequenceNo_x'])
    test2 = test2[test2['diff'] == 1].sort_values(by=['SecurityID', 'OfferApplSeqNum', 'sequenceNo_x'])
    test1 = test1.groupby(['SecurityID', 'BidApplSeqNum']).last().reset_index()
    test2 = test2.groupby(['SecurityID', 'OfferApplSeqNum']).last().reset_index()
    test = pd.concat([test1, test2])
    del test1
    del test2
    test['diff1'] = test['clockAtArrival_x'] - test['clockAtArrival_y']
    test['date'] = int(date)
    result += [test]
    del test
    print(date)
result = pd.concat(result).reset_index(drop=True)

C:\Users\win\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\win\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:64: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



20201014
20201015
20201016
20201019
20201020


In [6]:
result['count1'] = result['count1'] + 1
bins = [0, 1, 3, 5, 11, result['count1'].max()]
result['cat'] = pd.cut(result['count1'], bins)
# result.groupby('count1')['diff1'].describe().fillna(0).astype('int64')

In [13]:
add = result.groupby('cat')['diff1'].size().reset_index()
add['count%'] = ((add['diff1'] / add['diff1'].sum()) * 100).round(2)
re = result.groupby('cat')['diff1'].describe().fillna(0).astype('int64').reset_index()
re = pd.merge(re, add[['cat', 'count%']], on='cat')

In [14]:
add = result.groupby('cat')['diff1'].quantile(.9).reset_index()
add = add.rename(columns={'diff1':'90%'})
re = pd.merge(re, add[['cat', '90%']], on='cat')
re

,cat,count,mean,std,min,25%,50%,75%,max,count%,90%
0,"(0.0, 1.0]",66633305,4,57,0,2,3,4,17673,76.94,5.0
1,"(1.0, 3.0]",13591309,11,94,1,4,5,8,16727,15.69,17.0
2,"(3.0, 5.0]",3247314,21,128,2,6,11,18,16011,3.75,41.0
3,"(5.0, 11.0]",2279283,36,168,3,13,21,34,11803,2.63,81.0
4,"(11.0, 1426.0]",858481,112,293,7,37,68,135,23013,0.99,216.0


In [16]:
add = result.groupby('cat')['diff1'].quantile(.95).reset_index()
add = add.rename(columns={'diff1':'95%'})
add['95%'] = add['95%'].astype('int64')
re = pd.merge(re, add[['cat', '95%']], on='cat')
re

,cat,count,mean,std,min,25%,50%,75%,max,count%,90%,95%
0,"(0.0, 1.0]",66633305,4,57,0,2,3,4,17673,76.94,5,8
1,"(1.0, 3.0]",13591309,11,94,1,4,5,8,16727,15.69,17,36
2,"(3.0, 5.0]",3247314,21,128,2,6,11,18,16011,3.75,41,84
3,"(5.0, 11.0]",2279283,36,168,3,13,21,34,11803,2.63,81,111
4,"(11.0, 1426.0]",858481,112,293,7,37,68,135,23013,0.99,216,291


In [17]:
add = result.groupby('cat')['diff1'].quantile(.99).reset_index()
add = add.rename(columns={'diff1':'99%'})
add['99%'] = add['99%'].astype('int64')
re = pd.merge(re, add[['cat', '99%']], on='cat')
re

,cat,count,mean,std,min,25%,50%,75%,max,count%,90%,95%,99%
0,"(0.0, 1.0]",66633305,4,57,0,2,3,4,17673,76.94,5,8,59
1,"(1.0, 3.0]",13591309,11,94,1,4,5,8,16727,15.69,17,36,108
2,"(3.0, 5.0]",3247314,21,128,2,6,11,18,16011,3.75,41,84,136
3,"(5.0, 11.0]",2279283,36,168,3,13,21,34,11803,2.63,81,111,172
4,"(11.0, 1426.0]",858481,112,293,7,37,68,135,23013,0.99,216,291,558


In [18]:
re.groupby('cat')['count%', 'count', 'mean', 'std', 'min', '25%', '50%', '75%', '90%', '95%', '99%', 'max'].first()

,count%,count,mean,std,min,25%,50%,75%,90%,95%,99%,max
cat,,,,,,,,,,,,
"(0.0, 1.0]",76.94,66633305,4,57,0,2,3,4,5,8,59,17673
"(1.0, 3.0]",15.69,13591309,11,94,1,4,5,8,17,36,108,16727
"(3.0, 5.0]",3.75,3247314,21,128,2,6,11,18,41,84,136,16011
"(5.0, 11.0]",2.63,2279283,36,168,3,13,21,34,81,111,172,11803
"(11.0, 1426.0]",0.99,858481,112,293,7,37,68,135,216,291,558,23013


In [ ]:
add = result.groupby('cat')['diff1'].size().reset_index()
add['count%'] = ((add['diff1'] / add['diff1'].sum()) * 100).round(2)
re = result.groupby('cat')['diff1'].describe().fillna(0).astype('int64').reset_index()
re = pd.merge(re, add[['cat', 'count%']], on='cat')
add = result.groupby('cat')['diff1'].quantile(.9).reset_index()
add = add.rename(columns={'diff1':'90%'})
re = pd.merge(re, add[['cat', '90%']], on='cat')
add = result.groupby('cat')['diff1'].quantile(.95).reset_index()
add = add.rename(columns={'diff1':'95%'})
re = pd.merge(re, add[['cat', '95%']], on='cat')
add = result.groupby('cat')['diff1'].quantile(.99).reset_index()
add = add.rename(columns={'diff1':'99%'})
re = pd.merge(re, add[['cat', '99%']], on='cat')
re.groupby('cat')['count%', 'count', 'mean', 'std', 'min', '25%', '50%', '75%', '90%', '95%', '99%', 'max'].first()

In [24]:
result[(result['count1'] == 1) & (result['diff1'] == 17673)]

,Aggressive_side,ApplSeqNum_x,ApplSeqNum_y,BidApplSeqNum,ExecType,OfferApplSeqNum,SecurityID,TradeMoney,TradePrice,TradeQty,TransactTime,clockAtArrival_x,clockAtArrival_y,count1,cum_volume,diff,first,index,sequenceNo_x,sequenceNo_y,sum,diff1,date,cat
39037553,1,425137,425136,425136,F,424636,2411,67700000,67700,1000,93001440,1602811804972831,1602811804955158,1.0,846300,1,186.0,186,2559801,2558418,186.0,17673,20201016,"(0.0, 1.0]"


In [25]:
result[(result['count1'] == 2) & (result['diff1'] == 16727)]

,Aggressive_side,ApplSeqNum_x,ApplSeqNum_y,BidApplSeqNum,ExecType,OfferApplSeqNum,SecurityID,TradeMoney,TradePrice,TradeQty,TransactTime,clockAtArrival_x,clockAtArrival_y,count1,cum_volume,diff,first,index,sequenceNo_x,sequenceNo_y,sum,diff1,date,cat
71805748,1,13316705,13316703,13316703,F,13314751,2151,121020000,605100,200,130002690,1603170003728479,1603170003711752,2.0,18568741,1,49963.0,49964,74788822,74787695,49964.0,16727,20201020,"(1.0, 3.0]"


In [27]:
result[(result['count1'] == 5) & (result['diff1'] == 16011)]

,Aggressive_side,ApplSeqNum_x,ApplSeqNum_y,BidApplSeqNum,ExecType,OfferApplSeqNum,SecurityID,TradeMoney,TradePrice,TradeQty,TransactTime,clockAtArrival_x,clockAtArrival_y,count1,cum_volume,diff,first,index,sequenceNo_x,sequenceNo_y,sum,diff1,date,cat
54321638,1,419412,419407,419407,F,401982,987,179800000,179800,1000,93001190,1603071003690686,1603071003674675,5.0,279700,1,118.0,122,2652126,2651226,122.0,16011,20201019,"(3.0, 5.0]"


In [32]:
result[(result['count1'] == 10) & (result['diff1'] == 11803)]

,Aggressive_side,ApplSeqNum_x,ApplSeqNum_y,BidApplSeqNum,ExecType,OfferApplSeqNum,SecurityID,TradeMoney,TradePrice,TradeQty,TransactTime,clockAtArrival_x,clockAtArrival_y,count1,cum_volume,diff,first,index,sequenceNo_x,sequenceNo_y,sum,diff1,date,cat
30698391,2,359068,359058,267175,F,359058,2515,450850000,63500,7100,93000060,1602725401593799,1602725401581996,10.0,54800,1,0.0,9,2267289,2265146,9.0,11803,20201015,"(5.0, 11.0]"


In [3]:
test[(test['sequenceNo'] >= 2558418) & (test['SecurityID'] == 2411)][['SecurityID', 'sequenceNo', 'clockAtArrival', 'ApplSeqNum', 'OrderQty', 'Price', 'OrderType', 'Side', 'TradeQty', 'TradePrice', 'BidApplSeqNum', 'OfferApplSeqNum']].head(20)

,SecurityID,sequenceNo,clockAtArrival,ApplSeqNum,OrderQty,Price,OrderType,Side,TradeQty,TradePrice,BidApplSeqNum,OfferApplSeqNum
1374533,2411,2558418,1602811804955158,425136,1900.0,67800.0,2,1.0,NaN,NaN,NaN,NaN
417826,2411,2559801,1602811804972831,425137,NaN,NaN,NaN,NaN,1000.0,67700.0,425136.0,424636.0
1375306,2411,2559839,1602811804972867,425175,500.0,73300.0,2,2.0,NaN,NaN,NaN,NaN
1375311,2411,2559844,1602811804972872,425180,500.0,67200.0,2,1.0,NaN,NaN,NaN,NaN
1376377,2411,2561406,1602811804979847,425656,1000.0,67800.0,2,1.0,NaN,NaN,NaN,NaN
1376379,2411,2561408,1602811804979850,425658,100.0,67800.0,2,1.0,NaN,NaN,NaN,NaN
1376518,2411,2561620,1602811804980269,425747,500.0,65300.0,2,1.0,NaN,NaN,NaN,NaN
1377546,2411,2563181,1602811804991824,426026,2000.0,65000.0,2,1.0,NaN,NaN,NaN,NaN
1378936,2411,2565416,1602811805000413,426484,2000.0,67800.0,2,1.0,NaN,NaN,NaN,NaN
1378941,2411,2565422,1602811805000431,426490,200.0,67700.0,2,1.0,NaN,NaN,NaN,NaN


In [6]:
mdTradeLog[(mdTradeLog['sequenceNo'] >= 2265146) & (mdTradeLog['SecurityID'] == 2515)].head(20)

,clockAtArrival,sequenceNo,TransactTime,SecurityID,ApplSeqNum,cum_volume,ExecType,TradePrice,TradeQty,TradeMoney,BidApplSeqNum,OfferApplSeqNum
311949,1602725401581997,2265147,93000060,2515,359059,18900,F,63600,1800,114480000,172554,359058
311950,1602725401581999,2265148,93000060,2515,359060,28900,F,63600,10000,636000000,231064,359058
311951,1602725401581999,2265149,93000060,2515,359061,33200,F,63600,4300,273480000,247696,359058
311952,1602725401582000,2265150,93000060,2515,359062,34200,F,63500,1000,63500000,3832,359058
312555,1602725401593793,2267284,93000060,2515,359063,34900,F,63500,700,44450000,165684,359058
312556,1602725401593795,2267285,93000060,2515,359064,35600,F,63500,700,44450000,201802,359058
312557,1602725401593796,2267286,93000060,2515,359065,37300,F,63500,1700,107950000,226177,359058
312558,1602725401593797,2267287,93000060,2515,359066,47300,F,63500,10000,635000000,260447,359058
312559,1602725401593798,2267288,93000060,2515,359067,47700,F,63500,400,25400000,260591,359058
312560,1602725401593799,2267289,93000060,2515,359068,54800,F,63500,7100,450850000,267175,359058


In [1]:
import pandas as pd
import numpy as np
import glob
import os
import datetime

date_list = ['20201016']  
result = []
for d in date_list:
    date = d
    
    readPath = 'A:\\rawData'
    mdOrderLogPath = glob.glob(os.path.join(readPath, 'logs_%s_zs_92_01***'%date, 'mdOrderLog***.csv'))[-1]
    mdTradeLogPath = glob.glob(os.path.join(readPath, 'logs_%s_zs_92_01***'%date, 'mdTradeLog***.csv'))[-1]
    
    mdOrderLog = pd.read_csv(mdOrderLogPath)
    mdOrderLog['OrderType'] = mdOrderLog['OrderType'].astype(str)
    mdOrderLog = mdOrderLog[['clockAtArrival', 'sequenceNo', 'TransactTime', 'SecurityID', 'ApplSeqNum', 'Side',
                             'OrderType', 'Price', 'OrderQty']]
    
    mdTradeLog = pd.read_csv(mdTradeLogPath, encoding='utf-8')
    mdTradeLog['ExecType'] = mdTradeLog['ExecType'].astype(str)
    mdTradeLog['volumeThisUpdate'] = np.where(mdTradeLog['ExecType'] == 'F', mdTradeLog['TradeQty'], 0)
    mdTradeLog['cum_volume'] = mdTradeLog.groupby(['SecurityID'])['volumeThisUpdate'].cumsum()
    mdTradeLog = mdTradeLog[['clockAtArrival', 'sequenceNo', 'TransactTime', 'SecurityID', 'ApplSeqNum', 'cum_volume',
                             'ExecType', 'TradePrice', 'TradeQty', 'TradeMoney', 'BidApplSeqNum', 'OfferApplSeqNum']]
    mdTradeLog = mdTradeLog[mdTradeLog['ExecType'] == 'F']
    mdTradeLog = mdTradeLog[mdTradeLog['SecurityID'] < 600000]
    mdTradeLog = mdTradeLog[(mdTradeLog['TransactTime'] >= 92900000) & (mdTradeLog['TransactTime'] <= 145655000)]

C:\Users\win\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
